# Flower 연합학습 전략 구현을 통한 FedAvg 넘어서기

Flower를 활용한 KAICD 연합학습 강좌 2부에 오신 것을 환영합니다!

이번 튜토리얼에서는 지난 시간에 구축한 연합학습 시스템을 커스터마이징 할 것입니다(이번에도 FLower와 PyTorch를 사용합니다!).

Part 1에서는 PyTorch를 활용하여 모델 훈련 파이프라인과 데이터 불러오기를 구성합니다.

Part 2에서는 Flower를 활용하여 Part 1에서 구현한 PyTorch 기반 파이프라인을 가지고 연합학습을 진행합니다.

## 사전준비

시작하기에 앞서, Google Colab GPU 가속 설정을 확인해야 합니다.

`런타임 > 런타임 유형 변경 > 하드웨어 가속: GPU > 저장`

### 의존성 설치

다음으로 필요한 패키지를 설치하고 가져옵니다.

In [ ]:
!pip install torch==1.9.0 torchvision==0.10.0 git+https://github.com/adap/flower.git@release/0.17#egg=flwr["simulation"]

from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE = "cpu"  # CPU로 실행하시려면 해당 코드를 살려주세요!
print(f"Training on {DEVICE}")

Google Colab에서 실행하고 런타임에 GPU 가속기가 있다면, 출력 결과에서 `Training on cuda:0` 문장을 확인할 수 있습니다.

### 데이터 불러오기

이제 CIFAR-10 훈련 및 테스트 데이터 세트를 불러오고, 10개의 작은 데이터 세트로 분할하여(각각 훈련 및 검증 데이터 세트로 분할) `DataLoader`로 포장합니다:

In [ ]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):
    # CIFAR-10 데이터세트를 다운로드하고 전처리합니다(훈련 및 테스트).
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # 훈련 데이터세트를 10개로 나누고 개별 데이터세트를 시뮬레이션 합니다.
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # 분할된 각 파티션마다 훈련/검증 데이터세트로 나누고 데이터로더 인스턴스를 생성합니다.
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

### 모델 훈련/평가

기본적인 모델을 정의하고, 훈련 및 테스트 함수를 작성합니다:

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(testloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

## 연합학습 전략 커스터마이징

### Flower 클라이언트

Flower 클라이언트를 구현하기 위해 `flwr.client.NumPyClient`의 하위 클래스를 생성하고,

세 가지 함수(`get_parameters`, `fit`, `evaluate`)를 구현합니다:

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## 사용자 정의 전략 수립

슬슬 튜토리얼 노트북이 친숙해질 때가 되었습니다.

이제 Flower의 새로운 특징들을 소개하는 다음 단계로 넘어가기 위한 준비가 되었습니다!

### 서버 파라미터 초기화

Flower는 기본적으로 하나의 임의 클라이언트에 초기 파라미터를 요청하여 전역 모델을 초기화합니다.

그러나, 대부분의 경우 파라미터 초기화에 대한 더 많은 제어를 필요로 합니다.

따라서 Flower는 초기 파라미터를 전략에 직접 전달할 수 있는 기능을 구현해놨습니다:

In [ ]:
# 모델 인스턴스를 생성하고 파라미터를 불러옵니다.
net = Net()
params = get_parameters(Net())

# 서버 모델 파라미터를 초기화하기 위해 연합학습 전략을 수립합니다.
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(params),
)

# 시뮬레이션을 시작합니다.
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # 3번의 라운드 진행
    strategy=strategy,
)

`FedAvg` 전략에 `initial_parameters`를 전달하면 Flower는 클라이언트에게 초기 파라미터를 요청하지 않습니다.

로그를 자세히 살펴보면, `FlowerClient.get_parameters` 함수에 대한 통신이 전혀 이루어지지 않았음을 알 수 있습니다.

### 커스터마이징 전략으로 연합학습 시작

우리는 이전에도 `start_simulation`을 마주한 적이 있습니다.

해당 함수는 `FlowerClient` 인스턴스를 생성하고 시뮬레이션하는데 사용된 `client_fn`, `num_client`, `strategy` 등 많은 변수를 받아들입니다.

연합학습 전략은 연합학습의 접근법 및 알고리즘으로 정의할 수 있습니다.

예를 들어, FedAvg 또는 FedAdagrad를 캡슐화하는 방법이 있죠.

이번에는 다른 전략을 사용해보도록 하겠습니다.

In [ ]:
# FedAdam 연합학습 전략 수립
strategy=fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
)

# 시뮬레이션 시작
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # 3번의 라운드 진행
    strategy=strategy,
)

## 서버 파라미터 평가

Flower는 서버 또는 클라이언트 측의 집계된 모델을 평가할 수 있습니다.

서버와 클라이언트 측 평가는 어떤 면에선 비슷하다고 할 수 있지만, 다른 면도 존재합니다.

**중앙집중형 평가**는 간단한 개념입니다.

중앙집중형 기계학습에서 평가하는 것과 같은 방식으로 작동합니다.

평가 목적으로 사용할 수 있는 서버 데이터 세트가 있다면, 제일 좋겠죠?

위의 경우에는 클라이언트에게 모델을 보내지 않고도 훈련 각 라운드 후에 새로 집계된 모델을 평가할 수 있습니다.

또한, 전체 평가 데이터 세트를 항상 사용할 수 있다는 장점이 있습니다.

**연합 평가**는 보다 복잡하지만, 더 강력한 측면이 있습니다:

1. 중앙집중형 데이터 세트가 필요하지 않습니다.
2. 더 큰 데이터 세트에 걸쳐 모델을 평가할 수 있게 해줍니다.
3. 더 현실적인 평가 결과를 산출할 수 있습니다.

사실, 우리가 대표적인 평가 결과를 얻기를 원한다면 많은 시나리오들이 **연합 평가**를 사용하길 요구합니다.

하지만, 이런 강력한 기능을 사용하는 데에는 많은 비용이 듭니다.

우선 클라이언트 측에서 평가하기 시작하면, 우리는 평가 데이터 세트가 연속적인 학습에 따라 종종 변화한다는 것을 인지해야 합니다.

모델을 변경하지 않더라도, 클라이언트가 항상 연결되어 있지 않고 각 클라이언트의 데이터 세트가 변경될 수 있기 때문에

시간이 지남에 따라 평가 결과가 변동(불안정한 평가 결과)하는 것을 관찰할 수 있습니다.

우리는 `FlowerClient`에서 평가 함수를 실행함으로써, 클라이언트 측에서 연합 평가가 어떻게 작용하는지 살펴봤습니다.

이제 서버 측에서 평가할 수 있는 방법을 살펴보겠습니다:

In [ ]:
# `evaluate` 함수는 매 라운드마다 Flower로부터 호출됩니다.
def evaluate(
    weights: fl.common.Weights,
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net()
    valloader = valloaders[0]
    set_parameters(net, weights)  # 가장 최신 파라미터로 업데이트
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
    eval_fn=evaluate,  # Pass the evaluation function
    # eval_fn=get_evaluation_fn(),  # Pass the evaluation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # 3번의 라운드 진행
    strategy=strategy,
)

## Part 4: 서버에서 클라이언트로 값 전송

때때로 서버 측에서 클라이언트 측 실행(훈련 및 평가)를 구성해야 할 때가 있습니다.

이에 대한 한 가지 예로, 클라이언트에게 특정 수의 local epoch을 추적하도록 요청하는 서버가 있습니다.

Flower는 딕셔너리 구조를 사용하여 서버에서 클라이언트로 구성 값을 전송하는 방법을 제공합니다.

클라이언트가 `fit`과정에서 `config` 파라미터를 통해 서버로부터 값을 전달받는 사례를 살펴보겠습니다(`config`는 평가에서도 마찬가지로 적용됩니다).

`fit` 함수는 `config` 파라미터를 통해 구성 사전(configuration dictionary)를 전달받은 다음, 해당 사전에서 값을 읽어들일 수 있습니다.

이 예제에서는 `current_round`와 `local epcohs`를 읽어들이고, 이 값들을 사용하여 로그를 갱신하고 지역 훈련 단계(local training epochs)를 구성합니다:

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        current_round = config["current_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {current_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

그렇다면 어떻게 이 구성 사전을 서버에서 클라이언트로 전송할 수 있을까요?

이를 위해 Flower 전략은 내장된 함수를 제공하며, 서버 측 평가와 유사하게 작동합니다.

전략은 연합학습 내 모든 라운드마다 이 함수를 호출합니다:

In [ ]:
def fit_config(rnd: int):
    """Return training configuration dict for each round.
    
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "current_round": rnd,  # The current round of federated learning
        "local_epochs": 1 if rnd < 2 else 2,  # 
    }
    return config

다음으로, 시뮬레이션을 시작하기 전에 이 함수를 `strategy`에 전달합니다:

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
    eval_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # Just three rounds
    strategy=strategy,
)

여기서 볼 수 있듯이, 클라이언트 로그에는 현재 연합 학습의 라운드(구성 사전에서 읽어들인 것)가 포함되어 있습니다.

또한 연합학습 과정에서 첫 번째, 두 번째 라운드에는 1 epoch 만큼 학습하고, 세 번째 라운드에는 2 epoch만큼 학습하도록 클라이언트 훈련을 구성할 수 있습니다.

클라이언트는 임의 값을 서버에 반환할 수도 있습니다.

이를 위해 `fit` 및 `evaluate` 과정에서 사전을 반환합니다.

이 개념은 명시적으로 언급하진 않았지만, 이 튜토리얼 전체에 걸쳐 보고 사용해왔습니다.

`FlowerClient`는 커스텀 키/값 쌍이 내재되어 있는 사전을 `evaluate`에서 세 번째 반환 값으로 반환합니다.

## 연합학습 스케일링

이제 마지막 단계입니다!

Flower를 사용하여 어떻게 많은 수의 클라이언트들을 실험할 수 있는지 살펴보겠습니다.

**우린 할 수 있습니다!**

In [ ]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

우리는 1,000개의 파티션을 가지고 있습니다.

각 파티션은 45개의 훈련 데이터 및 5개의 검증 데이터를 가지고 있습니다.

각 클라이언트의 훈련 데이터가 상당히 적다는 점을 감안할 때, 모델을 조금 더 훈련시켜야하므로, 클라이언트들이 3번의 지역 훈련을 수행하도록 구성해야 합니다.

또한, 각 라운드에서 훈련에 선발된 클라이언트의 비율(매 라운드마다 1,000명의 클라이언트가 참여하질 않길 바라며)`fraction_fit`을 0.1로 조정합니다.

이는 훈련 가능한 전체 클라이언트의 10%만 선정한다는 의미입니다:

In [ ]:
def fit_config(rnd: int):
    config = {
        "current_round": rnd,
        "local_epochs": 3,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.05,  # Select 50 clients in each round
    fraction_eval=0.1,
    min_fit_clients=50,
    min_eval_clients=100,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=10,
    strategy=strategy,
)

## 요약

이번 튜토리얼에서는 연합학습 전략을 커스터마이징하고, 다른 전략도 선택해보며 서버 파라미터를 초기화하고,

서버 측의 모델도 평가해봄으로써 시스템을 점진적으로 향상시킬 수 있는 방법을 살펴봤습니다.

**정말 적은 양의 코드에 꽤 많은 힘이 있습니다!**